# Notebook for Toronto Neighborhoods project

In [55]:
# imports
import pandas as pd
import numpy as np
import lxml
import requests
import folium
import os
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.cm as cm
import matplotlib.colors as colors

### 1. Read table from wikipedia

In [8]:
# get table from wikipedia
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
neigh = tables[0]
neigh.rename(columns = {'Postal Code': 'PostalCode'}, inplace=True)

In [9]:
# subset to Borough != 'Not assigned'
mask = neigh.Borough != 'Not assigned'
neigh = neigh[mask].reset_index(drop=True)
neigh.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
# check for Neighbourhood == 'Not assigned'
print(neigh[neigh.Neighbourhood == 'Not assigned'].shape)
print('All Neighbourhoods assigned!')

(0, 3)
All Neighbourhoods assigned!


In [11]:
# check for duplicate PostalCodes
print(neigh['PostalCode'].value_counts().max())
print('No duplicate PostalCodes found!')

1
No duplicate PostalCodes found!


In [12]:
# print the dataframe shape
print(neigh.shape)

(103, 3)


### 2. Add geocoding info to the neighborhoods

In [13]:
# note: the geocoder package did not work so will use the CSV file
geo = pd.read_csv('..\..\data\Geospatial_Coordinates.csv')
geo.rename(columns = {'Postal Code': 'PostalCode'}, inplace=True)
neigh_geo = neigh.merge(geo, on='PostalCode')
neigh_geo.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [23]:
# visualize postal codes
latitude = neigh_geo.Latitude.mean()
longitude = neigh_geo.Longitude.mean()
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, postalcode, borough, neighborhood in zip(neigh_geo['Latitude'], 
                                                       neigh_geo['Longitude'], 
                                                       neigh_geo['PostalCode'], 
                                                       neigh_geo['Borough'], 
                                                       neigh_geo['Neighbourhood']):
    label = '{}, {}, {}'.format(neighborhood, borough, postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [24]:
# subset to boroughs containing "Toronto"
mask = neigh_geo.Borough.str.contains('Toronto')
toronto_geo = neigh_geo[mask].reset_index(drop=True)
print(toronto_geo.shape)
toronto_geo.head()

(39, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [27]:
# visualize postal codes where borough contains "Toronto"
latitude = toronto_geo.Latitude.mean()
longitude = toronto_geo.Longitude.mean()
map_toronto_geo = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, postalcode, borough, neighborhood in zip(toronto_geo['Latitude'], 
                                                       toronto_geo['Longitude'], 
                                                       toronto_geo['PostalCode'], 
                                                       toronto_geo['Borough'], 
                                                       toronto_geo['Neighbourhood']):
    label = '{}, {}, {}'.format(neighborhood, borough, postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_geo)  
    
map_toronto_geo

### 3. Analyze and cluster neighborhoods

In [28]:
# set foursquare credentials and version
CLIENT_ID = os.getenv('FOURSQUARE_CLIENT_ID')
CLIENT_SECRET = os.getenv('FOURSQUARE_CLIENT_SECRET')
VERSION = '20180605'
LIMIT = 100

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues(names=toronto_geo['PostalCode'],
                                 latitudes=toronto_geo['Latitude'],
                                 longitudes=toronto_geo['Longitude']
                                )
print("All done!")

All done!


In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(1594, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


In [33]:
# derive features with one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

postal_code_counts = toronto_onehot.PostalCode.value_counts().reset_index()
postal_code_counts.columns = ['PostalCode', 'Count']
print(postal_code_counts[['Count']].quantile([0,0.05,0.1,0.25,0.33,0.5,0.67,0.75,0.9,0.95,1]))
toronto_hot_counts = toronto_onehot.merge(postal_code_counts, on='PostalCode')

print(toronto_hot_counts.shape)
toronto_hot_counts.head()

       Count
0.00    2.00
0.05    2.90
0.10    4.00
0.25   16.00
0.33   18.54
0.50   31.00
0.67   43.92
0.75   60.00
0.90  100.00
0.95  100.00
1.00  100.00
(1594, 235)


,PostalCode,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Count
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,45
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,45
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,45
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,45
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,45


In [34]:
# group by postal code and take mean by venue type
toronto_grouped = toronto_hot_counts.groupby('PostalCode').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(39, 235)


,PostalCode,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Count
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.20000,0.0,0.0,0.0,0.0,0.0,0.000000,5
1,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,...,0.02381,0.0,0.02381,0.0,0.0,0.0,0.0,0.0,0.023810,42
2,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,19
3,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054054,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.027027,37
4,M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,3


In [71]:
# define function to sort venue type by frequency of occurence
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:-1]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [73]:
# create the dataframe containint top ten venue types for each postal code
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Pub,Park,Neighborhood,Trail,Health Food Store,Yoga Studio,Dessert Shop,Diner,Discount Store,Distribution Center
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,M4L,Fast Food Restaurant,Gym,Park,Brewery,Sandwich Place,Restaurant,Pub,Pizza Place,Coffee Shop,Movie Theater
3,M4M,Coffee Shop,Brewery,Gastropub,Café,Bakery,American Restaurant,Yoga Studio,Convenience Store,Cheese Shop,Clothing Store
4,M4N,Bus Line,Park,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [74]:
# standardize values
mask = toronto_grouped.Count > 19
X = toronto_grouped[mask].drop('PostalCode', 1)
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)

In [80]:
# determine best number of clusters
toronto_grouped_clustering = cluster_dataset
n_clus = [2,3,4,5,6,7]

for k in n_clus:
    print("K: ",k)
    # run k-means clustering
    print("Clusters:")
    kmeans = KMeans(n_clusters=k).fit(toronto_grouped_clustering)
    # check cluster labels generated for each row in the dataframe
    print(pd.DataFrame(kmeans.labels_).value_counts())
    # Calculate silhouette_score
    print("Silhouette: ",silhouette_score(toronto_grouped_clustering, kmeans.labels_),"\n")

K:  2
Clusters:
0    15
1    10
dtype: int64
Silhouette:  0.03162931266365168 

K:  3
Clusters:
1    14
2    10
0     1
dtype: int64
Silhouette:  0.03332112996986923 

K:  4
Clusters:
1    13
2    10
3     1
0     1
dtype: int64
Silhouette:  0.041469848231013816 

K:  5
Clusters:
1    11
4     6
2     6
3     1
0     1
dtype: int64
Silhouette:  0.027205888276622757 

K:  6
Clusters:
5    12
1     4
0     4
4     3
3     1
2     1
dtype: int64
Silhouette:  0.010534614382957677 

K:  7
Clusters:
3    12
1     5
4     4
6     1
5     1
2     1
0     1
dtype: int64
Silhouette:  0.02028035837978671 



In [81]:
# run k-means clustering
kclusters = 4
kmeans = KMeans(n_clusters=kclusters,random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(pd.DataFrame(kmeans.labels_).value_counts())

# Calculate silhouette_score
print("Silhouette: ",silhouette_score(toronto_grouped_clustering, kmeans.labels_),"\n")

1    9
3    6
0    6
2    4
dtype: int64
Silhouette:  0.023548988182787434 



In [82]:
# add clustering labels
labeled = neighborhoods_venues_sorted[mask]
labeled.head()
labeled.insert(0, 'Cluster Labels', kmeans.labels_)
labeled.head()

,Cluster Labels,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,3,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
3,1,M4M,Coffee Shop,Brewery,Gastropub,Café,Bakery,American Restaurant,Yoga Studio,Convenience Store,Cheese Shop,Clothing Store
6,0,M4R,Coffee Shop,Clothing Store,Shoe Store,Seafood Restaurant,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Yoga Studio,Sporting Goods Shop
7,1,M4S,Pizza Place,Sandwich Place,Dessert Shop,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Toy / Game Store,Greek Restaurant
11,1,M4X,Coffee Shop,Restaurant,Café,Pub,Italian Restaurant,Bakery,Chinese Restaurant,Pizza Place,Park,Sandwich Place


In [83]:
# add clustering labels to toronto_geo
toronto_geo_merged = toronto_geo.merge(labeled, on='PostalCode')
print(toronto_geo_merged.shape)
print(toronto_geo_merged['Cluster Labels'].value_counts())
toronto_geo_merged.head()

(25, 16)
1    9
3    6
0    6
2    4
Name: Cluster Labels, dtype: int64


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Portuguese Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Bubble Tea Shop,Electronics Store,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub,Restaurant,Clothing Store,Moroccan Restaurant,Cosmetics Shop,Creperie
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Beer Bar,Farmers Market,Seafood Restaurant,Cheese Shop,Park,Beach


In [84]:
# create map
latitude = toronto_geo_merged.Latitude.mean()
longitude = toronto_geo_merged.Longitude.mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_geo_merged['Latitude'], 
                                  toronto_geo_merged['Longitude'], 
                                  toronto_geo_merged['Neighbourhood'], 
                                  toronto_geo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 4. Examine Clusters

In [85]:
# cluster 0
mask0 = toronto_geo_merged['Cluster Labels'] == 0
toronto_geo_merged.loc[mask0, toronto_geo_merged.columns[[1] + list(range(5, toronto_geo_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gym,Clothing Store,Deli / Bodega,Thai Restaurant,Sushi Restaurant,Concert Hall,Cosmetics Shop
7,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Brewery,Fried Chicken Joint,Scenic Lookout,Restaurant,Italian Restaurant,Bar
10,Downtown Toronto,0,Coffee Shop,Hotel,Café,Seafood Restaurant,Italian Restaurant,Restaurant,Salad Place,American Restaurant,Japanese Restaurant,Bakery
12,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Deli / Bodega,Seafood Restaurant,Cocktail Bar
15,Central Toronto,0,Coffee Shop,Clothing Store,Shoe Store,Seafood Restaurant,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Yoga Studio,Sporting Goods Shop
23,Downtown Toronto,0,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,American Restaurant,Seafood Restaurant,Steakhouse,Asian Restaurant


In [86]:
# cluster 1
mask1 = toronto_geo_merged['Cluster Labels'] == 1
toronto_geo_merged.loc[mask1, toronto_geo_merged.columns[[1] + list(range(5, toronto_geo_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Portuguese Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Fried Chicken Joint
11,West Toronto,1,Café,Breakfast Spot,Coffee Shop,Climbing Gym,Burrito Place,Restaurant,Stadium,Italian Restaurant,Bar,Intersection
13,East Toronto,1,Coffee Shop,Brewery,Gastropub,Café,Bakery,American Restaurant,Yoga Studio,Convenience Store,Cheese Shop,Clothing Store
14,West Toronto,1,Café,Thai Restaurant,Mexican Restaurant,Music Venue,Arts & Crafts Store,Diner,Bar,Italian Restaurant,Bakery,Speakeasy
17,Central Toronto,1,Pizza Place,Sandwich Place,Dessert Shop,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Toy / Game Store,Greek Restaurant
18,Downtown Toronto,1,Café,Bookstore,Bar,Japanese Restaurant,Bakery,Pub,Dessert Shop,Restaurant,Sandwich Place,Sushi Restaurant
19,West Toronto,1,Sushi Restaurant,Café,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Yoga Studio,Food & Drink Shop,Latin American Restaurant,Indie Movie Theater
22,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Pub,Italian Restaurant,Bakery,Chinese Restaurant,Pizza Place,Park,Sandwich Place


In [87]:
# cluster 2
mask2 = toronto_geo_merged['Cluster Labels'] == 2
toronto_geo_merged.loc[mask2, toronto_geo_merged.columns[[1] + list(range(5, toronto_geo_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,2,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub,Restaurant,Clothing Store,Moroccan Restaurant,Cosmetics Shop,Creperie
4,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Beer Bar,Farmers Market,Seafood Restaurant,Cheese Shop,Park,Beach
20,Downtown Toronto,2,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Bakery,Bar,Grocery Store,Caribbean Restaurant,Gaming Cafe
21,Downtown Toronto,2,Coffee Shop,Seafood Restaurant,Cocktail Bar,Hotel,Italian Restaurant,Japanese Restaurant,Restaurant,Beer Bar,Café,Gym


In [88]:
# cluster 3
mask3 = toronto_geo_merged['Cluster Labels'] == 3
toronto_geo_merged.loc[mask3, toronto_geo_merged.columns[[1] + list(range(5, toronto_geo_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Bubble Tea Shop,Electronics Store,Ramen Restaurant
5,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Bubble Tea Shop,Burger Joint,Yoga Studio,Ramen Restaurant,Bike Rental / Bike Share
8,West Toronto,3,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Restaurant,Café,Men's Store,Asian Restaurant,Cuban Restaurant,Brewery,Record Shop
9,East Toronto,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
16,Central Toronto,3,Café,Sandwich Place,Coffee Shop,Pharmacy,BBQ Joint,Pizza Place,Pub,Middle Eastern Restaurant,Burger Joint,Donut Shop
24,Downtown Toronto,3,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Hotel,Fast Food Restaurant
